V2 pipeline. Pre process on frontera nd then submit directly to metashape job
    Step 1 rclone all files onto scratch

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import glob
import subprocess
import time


/home1/08531/dlforr/.local/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
aquisition_month="5_2019_Jan"
aquisition_date="Jan_08_2019"
aquisition_date_folder="1_2019_Jan_08"
drive_base="gcloud:2018_Yasuni_Drone_Mapping"

base_folder="/scratch1/08531/dlforr/001_image_preprocess"




First create the necesarry subfolders

In [3]:
os.chdir(base_folder)

if not os.path.exists(aquisition_date_folder):    
    os.mkdir(aquisition_date_folder)
    os.mkdir(aquisition_date_folder+"/Proccessed_Images")
    os.mkdir(aquisition_date_folder+"/Proccessed_Images/Phantom")
    os.mkdir(aquisition_date_folder+"/Reflectance")
    
    
    

Next clone everyting using rclone

1) copy refectance data

In [ ]:
rclone_path= "~/src/rclone-v1.57.0-linux-amd64/rclone"

rclone_source = drive_base  + "/" + aquisition_month + "/" + aquisition_date + "/Reflectance/"
rclone_dest = "./" + aquisition_date_folder+"/Reflectance/"

#print(rclone_source)
#print(rclone_dest)


command = rclone_path +" sync " +  rclone_source + " " + rclone_dest + " -v"
print(command)

!{command}


~/src/rclone-v1.57.0-linux-amd64/rclone sync gcloud:2018_Yasuni_Drone_Mapping/5_2019_Jan/Jan_08_2019/Reflectance/ ./1_2019_Jan_08/Reflectance/ -v
2022/08/19 15:54:38 INFO  : .DS_Store: Copied (new)
2022/08/19 15:54:39 INFO  : MAPIR/2019_January_08_Flight_1_Post_1_B.JPG: Copied (new)
2022/08/19 15:54:39 INFO  : MAPIR/2019_January_08_Flight_2_Pre_1_B.JPG: Copied (new)
2022/08/19 15:54:39 INFO  : MAPIR/2019_January_08_Flight_1_Post_1_A.RAW: Copied (new)
2022/08/19 15:54:40 INFO  : MAPIR/2019_January_08_Flight_2_Pre_1_A.RAW: Copied (new)
2022/08/19 15:54:40 INFO  : MAPIR/2019_January_08_Flight_3_Post_1_A.RAW: Copied (new)
2022/08/19 15:54:40 INFO  : MAPIR/2019_January_08_Flight_3_Post_1_B.JPG: Copied (new)
2022/08/19 15:54:41 INFO  : MAPIR/2019_January_08_Flight_4_Post_1_B.JPG: Copied (new)
2022/08/19 15:54:41 INFO  : MAPIR/2019_January_08_Flight_4_Post_1_A.RAW: Copied (new)
2022/08/19 15:54:41 INFO  : MAPIR/2019_January_08_Flight_4_Pre_1_A.JPG: Copied (new)
2022/08/19 15:54:42 INFO  : MAP

2022/08/19 15:55:05 INFO  : Stationary_Reflectance/2018_0108_111733_075.JPG: Copied (new)
2022/08/19 15:55:05 INFO  : Stationary_Reflectance/2018_0108_111834_076.JPG: Copied (new)
2022/08/19 15:55:05 INFO  : Stationary_Reflectance/2018_0108_111935_077.JPG: Copied (new)
2022/08/19 15:55:06 INFO  : Stationary_Reflectance/2018_0108_112036_078.JPG: Copied (new)
2022/08/19 15:55:06 INFO  : Stationary_Reflectance/2018_0108_112137_079.JPG: Copied (new)
2022/08/19 15:55:07 INFO  : Stationary_Reflectance/2018_0108_112238_080.JPG: Copied (new)
2022/08/19 15:55:07 INFO  : Stationary_Reflectance/2018_0108_112339_081.JPG: Copied (new)
2022/08/19 15:55:07 INFO  : Stationary_Reflectance/2018_0108_112440_082.JPG: Copied (new)
2022/08/19 15:55:07 INFO  : Stationary_Reflectance/2018_0108_112541_083.JPG: Copied (new)
2022/08/19 15:55:08 INFO  : Stationary_Reflectance/2018_0108_112844_086.JPG: Copied (new)
2022/08/19 15:55:08 INFO  : Stationary_Reflectance/2018_0108_112945_087.JPG: Copied (new)
2022/08/19

Copy Phantom processed images

In [ ]:
rclone_source = drive_base  + "/" + aquisition_month + "/" + aquisition_date + "/Proccessed_Images/Phantom/"
rclone_dest = "./" + aquisition_date_folder+"/Proccessed_Images/Phantom/"

command = rclone_path +" sync " +  rclone_source + " " + rclone_dest + " -v"
print(command)

!{command}

Now that all necessary data have been copied calculate reflectance and choose a base image

In [ ]:
os.chdir("/home1/08531/dlforr/github_dir/YFDP_Photogrametry_Workflow_V2_2021_12_23/code")
import functions_calculate_image_average_extract_coordinates as pipeline_functions


colors = {'Target':'tab:blue', 1:'tab:orange', 2:'tab:green', 3:'tab:red', 4:'tab:purple', 5:'tab:brown'}

def plot_dataset(images_path,reference_path,cam_type):
    cam_types = ['Mapir', 'Mapir', 'Stationary']
    if cam_type not in cam_types:
        raise ValueError("Invalid camera type. Expected one of: %s" % cam_types)

    images = pd.read_csv(images_path)
    images["file_name"] = images["file_name"].str[:-4]
    images = images.sort_values(["flight", "image"], ascending = (True, True))
    images["order_all"] = list(range(1,len(images["file_name"]+1)))
    phantom_reflectance["file_name"] = phantom_reflectance["file_name"].str[:-4]
    images = pd.read_csv(images_path)
    images["file_name"] = images["file_name"].str[:-4]
    reference_target = pd.read_csv(images_path)
    reference_target = reference_target.sort_values(["flight"], ascending = (True))
    reference_target["order_all"] = list(range(1,len(phantom_reflectance["file_name"])+1))
    reference_target["flight"] = "Target"
    combined = pd.concat([reference_target,reference_target])
    combined = combined.reset_index()
    combined["date"] = combined.date.astype(str)
    combined["date"] = [dt.datetime.strptime(d,"%Y:%m:%d %H:%M:%S") for d in combined["date"]]
    #{['Phantom':0, 'Mapir':1, 'Stationary':2]}
    
    #need to apply function to each type of camera and then plot at once
    combined.plot(x="date", y="mean_red", kind="scatter",c=combined['flight'].map(colors),title='Image Reflectance Per Flight', xlabel = "Time", ylabel = "Red Reflectance", ax=axis[{'Phantom':0, 'Mapir':1, 'Stationary':2}[cam_type]])

  

Work with Phantom Data:
Step 1: Extract image metadata and coordinates from Phantom Images and Calculate averages:
Step 2: Read in averages, Remove, file extension Sort images and create an overall order and plot change in reflectance over time/



In [ ]:
#general path to date folder
proj_base_path= base_folder + "/" + aquisition_date_folder
os.chdir(proj_base_path)

In [ ]:
phantom_path = './Proccessed_Images/Phantom/'

start_time = time.time()
if not os.path.exists(phantom_path + "image_mean_reflectance.csv"):    
    pipeline_functions.calc_image_average(phantom_path)

print("--- %s minutes ---" % int((time.time() - start_time)/60))


In [ ]:
Phantom_images = pd.read_csv(phantom_path + "image_mean_reflectance.csv")
Phantom_images = Phantom_images.sort_values(["flight", "image"], ascending = (True, True))
Phantom_images["order_all"] = list(range(1,len(Phantom_images["file_name"])+1))
Phantom_images


Read in Phantom Reflectance Target Images

In [ ]:
Phantom_Reflectance_path= proj_base_path + '/Reflectance/Phantom/'

if not os.path.exists(Phantom_Reflectance_path + "image_mean_reflectance.csv"):    
    pipeline_functions.calc_image_average(Phantom_Reflectance_path)
    

phantom_reflectance = pd.read_csv(Phantom_Reflectance_path + "image_mean_reflectance.csv")
phantom_reflectance = phantom_reflectance.sort_values(["flight"], ascending = (True))
phantom_reflectance["order_all"] = list(range(1,len(phantom_reflectance["file_name"])+1))
phantom_reflectance["flight"] = "Target"
phantom_reflectance



Combine Phantom Images with the reflectance target data

In [ ]:
phantom = pd.concat([phantom_reflectance,Phantom_images])
phantom = phantom.reset_index()
phantom["date"] = phantom.date.astype(str)
phantom["date"]= [dt.datetime.strptime(d,"%Y:%m:%d %H:%M:%S") for d in phantom["date"]]
phantom

Read in Stationary Reflectance
*note october 2nd the folders are messed up so you have to use this special function

In [ ]:
Station_path= proj_base_path + "/Reflectance/Stationary_Reflectance/"
print(Station_path)

start_time = time.time()
if not os.path.exists(Station_path + "image_mean_reflectance.csv"):
    pipeline_functions.calc_image_average_stationary(Station_path)
    print("--- %s minutes ---" % (time.time() - start_time))
    


In [ ]:
stationary_refectance = pd.read_csv(Station_path + "image_mean_reflectance.csv")
stationary_refectance = stationary_refectance.sort_values(["date"], ascending = (True))
stationary_refectance["order_all"] = list(range(1,len(stationary_refectance["file_name"])+1))

stationary_refectance["date"] = stationary_refectance.date.astype(str)
stationary_refectance["date"]= [dt.datetime.strptime(d,"%Y:%m:%d %H:%M:%S") for d in stationary_refectance["date"]]
stationary_refectance["flight"] = 1
stationary_refectance["image"] = stationary_refectance["order_all"]
stationary_refectance

Adjust Time Stamp values that were messed up due to cameras not being insync

Phantom = Correct
MAPIR = 3 Hours Early
Stationary = Correct


In [ ]:
hours_added = dt.timedelta(hours = 0)
phantom["date"] = phantom["date"]+hours_added

In [ ]:
phantom_target = phantom[phantom["flight"] == "Target"]
phantom_target

phantom_target.plot(x="date", y="mean_red", kind="scatter",c=phantom_target['flight'].map(colors),title='Phantom', xlabel = "Time", ylabel = "Red Reflectance")
# Make most of the ticklabels empty so the labels don't get too crowded



#ticklabels_1 = ['']*len(phantom.index)
# Every 4th ticklable shows the month and day
#ticklabels_1[::2] = [item.strftime('%H:%M') for item in phantom.date[::2]]
#ax_1.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels_1))
#date_form = DateFormatter('%H:%M')
#ax_1.xaxis.set_major_formatter(date_form)


In [ ]:
figure, axis = plt.subplots(2, 1)

# Phantom Data
ax_1 = phantom.plot(x="date", y="mean_red", kind="scatter",c=phantom['flight'].map(colors),title='Phantom', xlabel = "Time", ylabel = "Red Reflectance",ax=axis[0])
# Make most of the ticklabels empty so the labels don't get too crowded

#ticklabels_1 = ['']*len(phantom.index)
# Every 4th ticklable shows the month and day
#ticklabels_1[::2] = [item.strftime('%H:%M') for item in phantom.date[::2]]
#ax_1.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels_1))
date_form = DateFormatter('%H:%M')
ax_1.xaxis.set_major_formatter(date_form)

  
# stationary_refectance
ax_3 = stationary_refectance.plot(x="date", y="mean_red", kind="scatter",c=stationary_refectance['flight'].map(colors),title='Stationary Reflectance', xlabel = "Time", ylabel = "Red Reflectance",ax=axis[1])
# Make most of the ticklabels empty so the labels don't get too crowded
#ticklabels_3 = ['']*len(stationary_refectance.index)
# Every 4th ticklable shows the month and day
#ticklabels_3[::2] = [item.strftime('%H:%M') for item in stationary_refectance.date[::2]]
#ax_3.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels_3))
ax_3.xaxis.set_major_formatter(date_form)


plt.subplots_adjust(hspace = 1)



figure.patch.set_facecolor('white')
figure.set_figheight(10)
#plt.savefig(proj_base_path + 'Proccessed_Images/1_Raw_Reflectance_2018_Feb_28.png', bbox_inches='tight')


#Next Images have to be normalized:

Step 1: identify normalization images.

Step 2: Normalize data using the following bash/python code




In [ ]:
#Phantom Images
start_time = time.time()

sourceFolder = proj_base_path + "/Proccessed_Images/Phantom"
destImage_rel = "2019_Jan_15_Phantom_Flight_1_80.JPG"

command = 'sourceFolder=' + sourceFolder + '; ' + 'cd "${sourceFolder}"; ' + 'destImage_rel=' + destImage_rel + '; ' + 'destImage="${sourceFolder}/${destImage_rel}"; ' + 'ls *.JPG | xargs -n 1 -P 32 -I @ python3 /home1/08531/dlforr/github_dir/YFDP_Photogrametry_Workflow_V2_2021_12_23/code/normalize_single_image.py -sourceFolder $sourceFolder -destImage $destImage -image @'

!{command}
print(str(round((time.time() - start_time)/60,2))+ " minutes")


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import subprocess



In [ ]:
sourceFolder = proj_base_path + "/Proccessed_Images/Phantom/"
sourceFolder_cmd='sourceFolder=' + sourceFolder + ';'
sourceFolder_cmd

In [ ]:
sourceFolder = proj_base_path + "/Proccessed_Images/Phantom/"
command = 'sourceFolder=' + sourceFolder + '; ' + 'cd "${sourceFolder}"; ' + '''add_exif() {image=$1; outputname="${image/.JPG/_HSV_V_fixed.JPG}"; output="./output/${outputname}"; echo "${image}"; eval "/home1/08531/dlforr/src/Image-ExifTool-12.44/exiftool -q -q -overwrite_original_in_place -TagsFromFile ${image} $'-all:all>all:all' ${output}"}; ''' +  'export -f add_exif;' + 'parallel -j8 add_exif {} ::: *.JPG'
print(command)
#subprocess.call(command,shell=True)

Now that all normalized images have been created, we need to add back in the exif data to the normalized images with Exif tools.

First add the phantom images back in

In [ ]:
#Phantom Images
sourceFolder = proj_base_path + "/Proccessed_Images/Phantom/"

start_time = time.time()

command = 'sourceFolder=' + sourceFolder + '; ' + 'cd "${sourceFolder}"; ' + 'ls *.JPG | xargs -n 1 -P 32 -I {} /home1/08531/dlforr/github_dir/YFDP_Photogrametry_Workflow_V2_2021_12_23/code/simple_add_exif.sh ' + sourceFolder + ' {}'
!{command}
print("--- %s seconds ---" % (time.time() - start_time))





In [ ]:
#After images have been normalized. Here we create the same graph as above to see how well normalization worked


In [ ]:
os.chdir(proj_base_path)
phantom_path = './Proccessed_Images/Phantom/'

start_time = time.time()
if not os.path.exists(phantom_path +"output/" + "image_mean_reflectance.csv"):    
    pipeline_functions.calc_image_average(phantom_path,post_norm=True)
    
if not os.path.exists(phantom_path +"output/" + "images_metadata_gps.csv"):    
    pipeline_functions.extract_gps_metadat_multi_thred(phantom_path +"output/")
    
print("--- %s minutes ---" % int((time.time() - start_time)/60))
    
    
Phantom_images_post_norm = pd.read_csv(phantom_path + "output/" "image_mean_reflectance.csv")
Phantom_images_post_norm = Phantom_images_post_norm.sort_values(["flight", "image"], ascending = (True, True))
Phantom_images_post_norm["order_all"] = list(range(1,len(Phantom_images_post_norm["file_name"])+1))

Phantom_images_post_norm


In [ ]:
phantom_post_norm = pd.concat([phantom_reflectance,Phantom_images_post_norm])
phantom_post_norm = phantom_post_norm.reset_index()
phantom_post_norm["date"] = phantom_post_norm.date.astype(str)
phantom_post_norm["date"]= [dt.datetime.strptime(d,"%Y:%m:%d %H:%M:%S") for d in phantom_post_norm["date"]]


phantom_post_norm["GCC"] = phantom_post_norm["mean_green"]/(phantom_post_norm["mean_red"]+phantom_post_norm["mean_green"]+phantom_post_norm["mean_blue"])
phantom_post_norm

In [ ]:
figure, axis = plt.subplots(3, 1)

# Phantom Data
ax_1 = phantom.plot(x="date", y="mean_red", kind="scatter",c=phantom['flight'].map(colors),title='Phantom', xlabel = "Time", ylabel = "Red Reflectance",ax=axis[0])
# Make most of the ticklabels empty so the labels don't get too crowded
date_form = DateFormatter('%H:%M')
ax_1.xaxis.set_major_formatter(date_form)

# phantom_post_norm Data
ax_2 = phantom_post_norm.plot(x="date", y="mean_red", kind="scatter",c=phantom_post_norm['flight'].map(colors),title='Phantom Normalized', xlabel = "Time", ylabel = "Red Reflectance",ax=axis[1])
# Make most of the ticklabels empty so the labels don't get too crowded
ax_2.xaxis.set_major_formatter(date_form)
  
# stationary_refectance
ax_3 = stationary_refectance.plot(x="date", y="mean_red", kind="scatter",c=stationary_refectance['flight'].map(colors),title='Stationary Reflectance', xlabel = "Time", ylabel = "Red Reflectance",ax=axis[2])
# Make most of the ticklabels empty so the labels don't get too crowded
ax_3.xaxis.set_major_formatter(date_form)


plt.subplots_adjust(hspace = 1)

figure.patch.set_facecolor('white')
figure.set_figheight(10)
#plt.savefig(proj_base_path + 'Proccessed_Images/1_Raw_Reflectance_2018_Feb_28.png', bbox_inches='tight')

Now sync the folder back up to google drive
1) sync reference data

In [ ]:
rclone_path= "~/src/rclone-v1.57.0-linux-amd64/rclone"
os.chdir(proj_base_path)
rclone_source = "./Reflectance/"
rclone_dest = drive_base  + "/" + aquisition_month + "/" + aquisition_date + "/Reflectance/"


command = rclone_path +" sync " +  rclone_source + " " + rclone_dest + " -v"


!{command}

Now copy all phantom processed image data

In [ ]:
rclone_source = proj_base_path +"/Proccessed_Images/Phantom/"
rclone_dest = drive_base  + "/" + aquisition_month + "/" + aquisition_date + "/Proccessed_Images/Phantom/"


command = rclone_path +" sync " +  rclone_source + " " + rclone_dest + " -v"
print(rclone_dest)

!{command}

Next build project base folder in scractch and copy images


In [ ]:
import shutil
os.chdir("/scratch1/08531/dlforr/000_metashape_projects")

if not os.path.exists(aquisition_date_folder):    
    os.mkdir(aquisition_date_folder)
    os.mkdir(aquisition_date_folder + "/images")
         
src_dir= proj_base_path + "/Proccessed_Images/Phantom/output/"       
dst_dir= "/scratch1/08531/dlforr/000_metashape_projects/"+ aquisition_date_folder+ "/images/"
                
          
for jpgfile in os.listdir(src_dir):
    if jpgfile.endswith(".JPG"):
        shutil.copy(os.path.join(src_dir, jpgfile), dst_dir) 

         
